<a href="https://colab.research.google.com/github/algodigger/playground/blob/main/notebooks/VisualTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>